In [1]:
pip install keras_metrics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade --user pip

# New section

In [3]:
import tensorflow as tf
import numpy as np
#!pip install sentencepiece
from tensorflow import keras
import keras_metrics
import pandas as pd
import sentencepiece as spm

In [24]:
#df = pd.read_csv('pasangan_baik.csv')

In [25]:
#lines_1 = df['key'].to_list()
#lines_2 = df['value'].to_list()

In [4]:
with open('min-ind.txt','r',encoding='utf-8') as f:
  lines = f.read().split('\n')
with open('min-ind_train.txt','w',encoding='utf-8') as w:
  for line in lines:
    w.write(line+'\n')
lines_1 = []
lines_2 = []
with open('min-ind_train.txt','r',encoding='utf-8') as min:
  for line in min.readlines():
    split = line.split(',')
    if split!=['\n']:

      lines_1.append(split[0])
      lines_2.append(split[1])
# preprocess kata di seitap lines 
#f = lambda x: re.sub('\n$', '', x)
#lines_1 = [f(x) for x in lines_1]
#lines_2 = [f(x) for x in lines_2]

# preprocess drop duplicates 
#df = pd.DataFrame([lines_1, lines_2], index=['key', 'value']).T.drop_duplicates()
#lines_1 = df['key'].to_list()
#lines_2 = df['value'].to_list()

with open('min-ind_input.txt','w',encoding='utf-8') as enc_txt:
  for line in lines_1:
    enc_txt.write(line+'\n')
with open('min-ind_target.txt','w',encoding='utf-8') as tar_txt:
  for line in lines_2:
    tar_txt.write(line)




In [37]:
df.duplicated().sum()

0

In [7]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 76 kB 1.1 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
import re
from tqdm import tqdm

def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], list):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value
kamus = {}
for i, row in tqdm(df.iterrows()):
  key = row['key']
  append_value(kamus, key, row['value'])

13207it [00:00, 14576.24it/s]


In [39]:
kamus

{'nan': ['yang\n', 'maadoki apo sajo"\n'],
 'di': 'di\n',
 'dari': 'dari\n',
 'jo': 'dengan\n',
 'dan': 'dan\n',
 'ko': 'ini\n',
 'adolah': 'adalah\n',
 'langau': 'nyamuk\n',
 'pado': 'pada\n',
 'juo': 'juga\n',
 'bagian': ['bagian\n', 'bidang\n', 'divisi\n'],
 'marupokan': 'merupakan\n',
 'dalam': 'dalam\n',
 'indonesia': 'indonesia\n',
 'biasonyo': 'biasanya\n',
 'spesies': 'spesies\n',
 'untuak': 'untuk\n',
 'kelas': 'kelas\n',
 'famili': 'famili\n',
 'ordo': 'ordo\n',
 'filum': 'filum\n',
 'arthropoda': 'arthropoda\n',
 'insecta': 'insecta\n',
 'daun': 'daun\n',
 'larva': 'larva\n',
 'diptera': 'diptera\n',
 'mamakan': 'memakan\n',
 'herbivora': 'herbivora\n',
 'dek': 'karena\n',
 'sabagai': 'sebagai\n',
 'ka': 'ka\n',
 'kota': 'kota\n',
 'taun': 'tahun\n',
 'ado': 'ada\n',
 'iko': 'ini\n',
 'inyo': 'dia\n',
 'manjadi': 'menjadi\n',
 'sabuah': 'sebuah\n',
 'atau': 'atau\n',
 'kabupaten': 'kabupaten\n',
 'marupoan': 'merupakan\n',
 'urang': ['orang\n', 'kepala\n', 'tenaga\n'],
 'pro

In [5]:
#import sentencepiece as spm
model_inp = spm.SentencePieceTrainer.train("--input=min-ind_input.txt --model_prefix=input --vocab_size=100")
sp_inp = spm.SentencePieceProcessor()
sp_inp.load('input.model')
model_tar = spm.SentencePieceTrainer.train("--input=min-ind_target.txt --model_prefix=target --vocab_size=100")
sp_tar = spm.SentencePieceProcessor()
sp_tar.load('target.model')

True

In [6]:
### Prepare data ###
input_encode = []
target_encode = []
with open('min-ind_input.txt','r',encoding='utf-8') as inp:
  for line in inp:
    input_encode.append(sp_inp.encode_as_pieces(line))
with open('min-ind_target.txt','r',encoding='utf-8') as tar:
  for line in tar:
    target_encode.append(['\t']+sp_tar.encode_as_pieces(line)+['\n'])
 

In [7]:
## Create a dictionary for characters mapping ##
input_subwords = np.unique(np.array([tk for tok in input_encode for tk in tok]))
input_subwords = np.append(input_subwords," ")
target_subwords = np.unique(np.array([tk for tok in target_encode for tk in tok]))
target_subwords = np.append(target_subwords," ")
input_token_index = dict([(char,i) for i,char in enumerate(sorted(list(input_subwords)))])
target_token_index = dict([(char,i) for i,char in enumerate(sorted(list(target_subwords)))])

num_encoder_tokens= len(input_subwords)
num_decoder_tokens = len(target_subwords)

In [8]:
max_encoder_seq_length = max([len(words) for words in input_encode])
max_decoder_seq_length = max([len(words) for words in target_encode])

encoder_input_data = np.zeros((len(input_encode),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(target_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
 
for i,(enc_word,tar_word) in enumerate(zip(input_encode,target_encode)):
  for t,char in enumerate(enc_word):
    encoder_input_data[i,t,input_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,input_token_index[" "]] = 1.0 # encoding space at the end of encoder word for decoder input to reach to next word for encoding
  for t,char in enumerate(tar_word):
    decoder_input_data[i,t,target_token_index[char]] = 1.0
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
  decoder_target_data[i,t:,target_token_index[" "]] = 1.0

In [9]:
len(encoder_input_data)

13761

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
seed = 7
np.random.seed(seed)
kfold_index,test_index = train_test_split(list(range(len(encoder_input_data))),test_size=0.2,random_state=seed)
fold_var = 5
kf = KFold(n_splits = fold_var,random_state=seed, shuffle=True)
validation_accuracy = []
validation_loss = []
exac_match_accuracy = []
#validation_recall = []

In [11]:
encoder_input_data_k = pd.Series(list(encoder_input_data)).iloc[kfold_index].to_numpy()
decoder_input_data_k = pd.Series(list(decoder_input_data)).iloc[kfold_index].to_numpy()
decoder_target_data_k = pd.Series(list(decoder_target_data)).iloc[kfold_index].to_numpy()

encoder_input_data_k = np.array([list(en) for en in encoder_input_data_k])
decoder_input_data_k = np.array([list(en) for en in decoder_input_data_k])
decoder_target_data_k = np.array([list(en) for en in decoder_target_data_k])

encoder_input_data_test = pd.Series(list(encoder_input_data)).iloc[test_index].to_numpy()
decoder_input_data_test = pd.Series(list(decoder_input_data)).iloc[test_index].to_numpy()
decoder_target_data_test = pd.Series(list(decoder_target_data)).iloc[test_index].to_numpy()

encoder_input_data_test = np.array([list(en) for en in encoder_input_data_test])
decoder_input_data_test = np.array([list(en) for en in decoder_input_data_test])
decoder_target_data_test = np.array([list(en) for en in decoder_target_data_test])

In [12]:
batch_size = 64  # Batch size for training.
epochs = 25 # Number of epochs to train for.
latent_dim_1 = 128
latent_dim_2 = 512
latent_dim_3 = 512 # Latent dimensionality of the encoding space.
num_samples = 13761  # Number of samples to train on.

In [15]:
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(encoder_input_data_k)))):
  print('Iteration number : ', i+1)

  encoder_input_data_t = pd.Series(list(encoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data_k)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data_k)).iloc[val_index].to_numpy()

  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])

  x_train = [encoder_input_data_t, decoder_input_data_t]
  y_train = decoder_target_data_t

  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val

  encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
  encoder_outputs,states_h,states_c=keras.layers.LSTM(latent_dim_2, return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(encoder_inputs)
  #encoder_outputs,states_h,states_c=keras.layers.LSTM(latent_dim_2, return_state=True)(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  #state_h = keras.layers.Concatenate()([forward_h,forward_c])
  #state_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c] # hidden and cell state 

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, 
                                                    return_sequences=True, 
                                                    return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(decoder_inputs, initial_state=encoder_states)
  #decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, return_sequences=True, 
                                                    #return_state=True)(decoder_inputs, initial_state=encoder_states)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  #decoder_dropout = keras.layers.Dropout(0.8)
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_outputs = decoder_dropout(decoder_outputs)
  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # Compile Model
  model.compile(
      optimizer="adam", loss="categorical_crossentropy", 
      metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  # Fitting Model
  model_history = model.fit(
      x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data = (x_val,y_val)
  )

  # Save model
  model_name = 'model_lstm_256_{}-Fold-{}'.format(fold_var,i+1)
  model.save(model_name)

  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  model_inf = keras.models.load_model(model_name)
  encoder_inputs = model_inf.input[0]  # input_1
  encoder_outputs,states_h,states_c  = model_inf.layers[2].output  # lstm_1
  #states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  #states_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model_inf.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model_inf.layers[3]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model_inf.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_dropout = model_inf.layers[7]
  #decoder_outputs = decoder_dropout(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )
  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token

          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence

  # Evaluate Model
  results = model.evaluate(x_val, y_val, batch_size=batch_size)
  results = dict(zip(model.metrics_names,results))
  
  # Save Evaluation Result
  validation_accuracy.append(results['accuracy'])
  validation_loss.append(results['loss'])
  
 
  # Check Result Translation
  
  # Check Result Translation
  match = 0
  for seq_index in val_index:
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)[1:-1]
    #print("-",seq_index)
    #print("Input sentence:", lines_1[seq_index])
    #print("Decoded sentence:", decoded_sentence)
    #print("Target sentence:", lines_2[seq_index][:-1])
    exact_result = decoded_sentence == lines_2[seq_index][:-1]
   #print("Exact Match:", str(exact_result))
    match += exact_result
  print("Exact Match count : ", match)
  print("Tested Data count : ", len(val_index))
  print("Translation Accuracy : ", match/len(val_index))
  

Iteration number :  1
Epoch 1/25
138/138 [==============================] - 2s 13ms/step - loss: 1.6469 - accuracy: 0.6628 - recall_5: 0.5821 - precision_5: 0.9579 - val_loss: 1.3997 - val_accuracy: 0.6998 - val_recall_5: 0.6135 - val_precision_5: 0.9852
Epoch 2/25
138/138 [==============================] - 1s 9ms/step - loss: 1.2336 - accuracy: 0.7218 - recall_5: 0.6772 - precision_5: 0.9753 - val_loss: 1.1599 - val_accuracy: 0.7298 - val_recall_5: 0.6984 - val_precision_5: 0.9725
Epoch 3/25
138/138 [==============================] - 1s 9ms/step - loss: 1.0937 - accuracy: 0.7426 - recall_5: 0.6992 - precision_5: 0.9771 - val_loss: 1.0550 - val_accuracy: 0.7500 - val_recall_5: 0.6981 - val_precision_5: 0.9761
Epoch 4/25
138/138 [==============================] - 1s 9ms/step - loss: 0.9970 - accuracy: 0.7585 - recall_5: 0.7074 - precision_5: 0.9768 - val_loss: 0.9735 - val_accuracy: 0.7637 - val_recall_5: 0.7053 - val_precision_5: 0.9767
Epoch 5/25
138/138 [=============================

In [59]:
# Show Acciuracy Result
for i,val_acc in enumerate(validation_accuracy):
  print("Fold {} Accuracy : {}".format(i+1,val_acc))
print("Average Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(validation_accuracy), np.std(validation_accuracy)))

Fold 1 Accuracy : 0.7863069772720337
Fold 2 Accuracy : 0.7799652814865112
Fold 3 Accuracy : 0.7813535332679749
Fold 4 Accuracy : 0.7852027416229248
Fold 5 Accuracy : 0.782584011554718
Fold 6 Accuracy : 0.7840037941932678
Fold 7 Accuracy : 0.8991007804870605
Fold 8 Accuracy : 0.8984382152557373
Fold 9 Accuracy : 0.9003943800926208
Fold 10 Accuracy : 0.8996686935424805
Fold 11 Accuracy : 0.8994478583335876
Average Accuracy : 0.84% (+/- 0.06%)


In [60]:
#Prep Test Data
x_test = [encoder_input_data_test,decoder_input_data_test]
y_test = decoder_target_data_test

# Evaluate Model
results = model.evaluate(x_test, y_test, batch_size=batch_size)
results = dict(zip(model.metrics_names,results))

42/42 [==============================] - 0s 5ms/step - loss: 0.5227 - accuracy: 0.8999 - recall_15: 0.8882 - precision_15: 0.9243


In [61]:
seq_index = 100
print(lines_1[seq_index])
print(lines_2[seq_index])

mambuek
membuat



In [17]:
# Check Result Translation -> Test Data
list_match = []
count = 0
for seq_index in test_index:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence[1:-1]
    print("-",seq_index)
    print("Input sentence:", ''.join(lines_1[seq_index]))
    print("Decoded sentence:", ''.join(decoded_sentence))
    print("Target sentence:", lines_2[seq_index][:-1])
    exact_result = decoded_sentence == lines_2[seq_index][:-1]
    print("Exact Match:", ''.join(str(exact_result)))
    list_match.append(exact_result)
    count+=1
print("Exact Match count : ", sum(list_match))
print("Tested Data count : ", count)
print("Translation Accuracy : ", sum(list_match)/count)

- 3120
Input sentence: diagnosis
Decoded sentence: diagsi
Target sentence: diagnosis
Exact Match: False
- 3225
Input sentence: sah
Decoded sentence: sah
Target sentence: sah
Exact Match: True
- 8062
Input sentence: marancanokan
Decoded sentence: merencanakan
Target sentence: merencanakan
Exact Match: True
- 9861
Input sentence: tunangannyo
Decoded sentence: tundunya
Target sentence: tunangannya
Exact Match: False
- 5853
Input sentence: mambuko
Decoded sentence: membuka
Target sentence: membuka
Exact Match: True
- 3772
Input sentence: kaagamoan
Decoded sentence: kegagamaan
Target sentence: keagamaan
Exact Match: False
- 9768
Input sentence: aktifnyo
Decoded sentence: aktifnya
Target sentence: aktifnya
Exact Match: True
- 690
Input sentence: kristen
Decoded sentence: kresfik
Target sentence: kristen
Exact Match: False
- 10801
Input sentence: kaget
Decoded sentence: gete
Target sentence: kaget
Exact Match: False
- 1986
Input sentence: dilahiakan
Decoded sentence: dilahirkan
Target sentenc

In [ ]:
!zip -r model_lstm_256_5-Fold-5.zip model_lstm_256_5-Fold-5